# Document representation

In [1]:
!pip install -U "datasets<3.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.6.1 which is incompatible.


In [2]:
import sklearn.feature_extraction # Módulo de sklearn para extracción de características
import datasets # Biblioteca de manejo de conjuntos de datos para procesamiento de lenguaje natural
import numpy as np # Biblioteca de manejo de datos vectoriales
import pandas as pd # Biblioteca de manejo de conjuntos de datos
import spacy.lang.es # Biblioteca de procesamiento de lenguaje natural
import matplotlib.pyplot as plt # Biblioteca de visualización
import sklearn.pipeline # Módulo de sklearn para el desarrollo de flujos de trabajo

Cargamos el conjunto de datos del curso.

In [3]:
spanish_diagnostics = datasets.load_dataset('fvillena/spanish_diagnostics') # Cargamos las particiones de entrenamiento y prueba

Generating train split:   0%|          | 0/70000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Cargamos una lista de stopwords desde la biblioteca Spacy.

In [4]:
stopwords = spacy.lang.es.stop_words.STOP_WORDS # La biblioteca Spacy tiene una lista de stopwords en español

## Representación de documentos

Para poder trabajar con datos de texto, estos deben ser representados de una manera que pueda ser interpretada por los algoritmos de minería de texto. Típicamente se desea llegar a una matriz que tenga tantas filas como documentos tenga nuestro corpus y tantas columnas como características fueron extraídas desde el texto.

Revisaremos 2 métodos de extracción de características:

* Bag-of-words: Este método extrae la frecuencia de aparición de cada una de las palabras del documento y representa un documento como un vector de tantas dimensiones como palabras tenga el vocabulario.

* Term frequency - inverse document frequency (TF-IDF): Este método extrae la frecuencia de aparición de cada una de las palabras y la multiplica por el inverso de la frecuencia de aparición de la palabra en todos los documentos. También se representa cada documento como un vector de tantas dimensiones como palabras tenga el vocabulario.

### Bag of Words

es una técnica utilizada en el procesamiento del lenguaje natural para representar textos como conjuntos no ordenados de palabras, ignorando la estructura gramatical y el orden de las palabras en el texto. En este enfoque, cada documento se representa mediante un vector que cuenta la ocurrencia o frecuencia de las palabras en el documento.

El proceso de creación de una representación de Bolsa de Palabras consta de los siguientes pasos:

* Tokenización: El texto se divide en unidades más pequeñas, generalmente palabras. Cada palabra se considera un "token".

* Creación del vocabulario: Se construye un vocabulario único a partir de todos los tokens en el conjunto de documentos. Cada palabra del vocabulario se asigna a un índice único.

* Creación del vector de características: Para cada documento, se crea un vector de características que representa la frecuencia de cada palabra del vocabulario en el documento. El valor en cada posición del vector corresponde a la frecuencia de esa palabra en el documento.

<center>
    <img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*hLvya7MXjsSc3NS2SoLMEg.png" alt="medium">
</center>





Instanciamos un extractor de características Bag-of-words.

In [5]:
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    stop_words = list(stopwords), # Le pasamos la lista de stopwords para eliminarlas del vocabulario
    max_df = 0.05, # Eliminamos del vocabulario el 5% de palabras más frecuentes (stopwords específicas del corpus)
    min_df = 2 # Eliminamos del vocabulario las palabras que tienen una frecuencia menor a 2 (típicamente palabras malformadas)
)

Ajustamos el vectorizador sobre los textos del conjunto de prueba.

In [6]:
count_vectorizer.fit(spanish_diagnostics["train"]["text"])

CountVectorizer(max_df=0.05, min_df=2,
                stop_words=['tendrá', 'tus', 'días', 'casi', 'estas', 'estos',
                            'mías', 'está', 'podrá', 'debe', 'cuánta', 'uso',
                            'cuánto', 'varias', 'quien', 'varios', 'diferente',
                            'segun', 'usted', 'vamos', 'debido', 'cuándo',
                            'modo', 'sé', 'bajo', 'sin', 'dio', 'menos',
                            'cuatro', 'mío', ...])

Exploraremos cómo está representando nuestros documentos este vectorizador.

Este es un texto de ejemplo del corpus.

In [7]:
text_vectorized = count_vectorizer.transform(spanish_diagnostics["train"]["text"])

In [8]:
text_vectorized.shape

(70000, 13565)

In [9]:
spanish_diagnostics["train"]["text"][69983]

'- CARIES DENTINARIA PROFUNDA/  - Fundamento Clínico APS: caries profunda en relacion de la pieza dental 4.5 Caries de la dentina'

In [10]:
count_vectorizer.transform([spanish_diagnostics["train"]["text"][69983]]).toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [11]:
def get_word_scores(text,vectorizer):
    """A partir de un texto y un vectorizador retorna los puntajes asignados a cada palabra del texto"""
    feature_names = list({k: v for k, v in sorted(vectorizer.vocabulary_.items(), key=lambda item: item[1])}.keys()) # Vocabulario
    doc = vectorizer.transform([text]) # Vectorizamos el texto de entrada
    idxs = np.argwhere(doc)[:,1] # Extraemos los índices donde sí hay palabras representadas
    words = [feature_names[i] for i in idxs] # Extraemos las palabras asociadas a los índices extraídos
    scores = np.array(doc.todense())[0][idxs] # Extraemos los puntajes asociadas a los índices extraídos
    return list(reversed(sorted(zip(words,scores),key=lambda tup: tup[1]))) # Retornamos una lista de palabras y puntajes

Vemos que nuestro vectorizador le dio más peso a la palabra caries de nuestro documento porque es la palabra más frecuente y a a un grupo de palabras les asignó el mismo puntaje 1 porque cada una aprece 1 vez.

In [12]:
get_word_scores(spanish_diagnostics["train"]["text"][69983],count_vectorizer)

[('caries', np.int64(3)),
 ('profunda', np.int64(2)),
 ('relacion', np.int64(1)),
 ('fundamento', np.int64(1)),
 ('dentinaria', np.int64(1)),
 ('dentina', np.int64(1)),
 ('dental', np.int64(1))]

### TF-IDF

Es una técnica utilizada en el procesamiento del lenguaje natural para ponderar la importancia de un término (palabra) en un documento dentro de una colección de documentos. Consiste en dos componentes principales: TF (Frecuencia del Término) e IDF (Frecuencia Inversa del Documento).

* Frecuencia del Término (TF): La componente TF mide la frecuencia relativa de un término en un documento específico. El objetivo es asignar un mayor peso a los términos que aparecen con más frecuencia dentro del documento, ya que se considera que estos términos son más importantes.
La fórmula básica para calcular la TF es: $$TF = \frac{(\text{Número de veces que aparece el término en el documento})}{(\text{Número total de términos en el documento})}$$

    Ejemplo de TF: Supongamos que tenemos un documento que contiene la siguiente frase: "El perro juega en el parque". Si queremos calcular la frecuencia del término "perro" en este documento, contaríamos que aparece una vez. Si asumimos que hay un total de cinco términos en el documento, entonces la frecuencia del término "perro" sería $\frac{1}{6}$ = 0.16.


* Frecuencia Inversa del Documento (IDF): La componente IDF mide la importancia de un término en el contexto de una colección de documentos. Se basa en la suposición de que los términos menos comunes en la colección pueden ser más informativos que los términos comunes. La fórmula básica para calcular el IDF es: $$IDF = log(\frac{\text{Número total de documentos en la colección}}{\text{Número de documentos que contienen el término}})$$

    El IDF se calcula como el logaritmo del cociente entre el número total de documentos y el número de documentos que contienen el término dado. El logaritmo se aplica para suavizar la escala del IDF y evitar una sobrevaloración de términos muy raros.

    Ejemplo de IDF: Supongamos que tenemos una colección de documentos que contiene un total de 100 documentos. Si el término "perro" aparece en 50 de esos documentos, entonces el IDF se calcularía como log(100/50) = log(2) ≈ 0.301.


* TF-IDF: El TF-IDF combina la información de TF e IDF para calcular un peso final para cada término en un documento. Se calcula multiplicando la TF del término en el documento por su IDF en la colección. La fórmula para calcular el TF-IDF es: $$TF-IDF = TF * IDF$$

    El TF-IDF aumentará para los términos que aparezcan con frecuencia en un documento específico (alta TF) y sean menos comunes en la colección en su conjunto (alta IDF).

    Ejemplo de TF-IDF: Supongamos que queremos calcular el TF-IDF para el término "perro" en un documento específico. Si el valor de TF es 0.2 (como en el ejemplo anterior) y el valor de IDF es 0.301 (como en el ejemplo IDF), entonces el valor de TF-IDF sería 0.2 * 0.301 = 0.0602.

Ajustamos un vectorizador que utiliza TF-IDF y lo ajustamos.

In [13]:
tfidf_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(
    stop_words = list(stopwords),
    max_df = 0.05,
    min_df = 2
)
tfidf_vectorizer.fit(spanish_diagnostics["train"]["text"])

TfidfVectorizer(max_df=0.05, min_df=2,
                stop_words=['tendrá', 'tus', 'días', 'casi', 'estas', 'estos',
                            'mías', 'está', 'podrá', 'debe', 'cuánta', 'uso',
                            'cuánto', 'varias', 'quien', 'varios', 'diferente',
                            'segun', 'usted', 'vamos', 'debido', 'cuándo',
                            'modo', 'sé', 'bajo', 'sin', 'dio', 'menos',
                            'cuatro', 'mío', ...])

Podemos observar que caries sigue siendo la palabra con el mayor puntaje. Pero vemos que todas las palabras tienen un puntaje distinto, en donde destacamos que la palabra fundamento tiene el menor puntaje. Intuitivamente podemos darnos cuenta que esta representación es mejor porque la palabra fundamento no nos aporta mucha información en el documento.

In [14]:
get_word_scores(spanish_diagnostics["train"]["text"][69983],tfidf_vectorizer)

[('caries', np.float64(0.5841232685802327)),
 ('profunda', np.float64(0.5052614123771013)),
 ('dentina', np.float64(0.3636207767489411)),
 ('relacion', np.float64(0.30054370920913476)),
 ('dentinaria', np.float64(0.27911008594273823)),
 ('dental', np.float64(0.25831112531154554)),
 ('fundamento', np.float64(0.19062330472026243))]

Vectorizamos los textos de nuestros conjuntos de entrenamiento y prueba. con el método CountVectorizer.transform()

In [15]:
text_vectorized_train = count_vectorizer.transform(spanish_diagnostics["train"]["text"])
text_vectorized_test = count_vectorizer.transform(spanish_diagnostics["test"]["text"])
feature_names = list({k: v for k, v in sorted(count_vectorizer.vocabulary_.items(), key=lambda item: item[1])}.keys())

La forma de nuestra matriz es de (cantidad de documentos en el conjunto, tamaño del vocabulario)

In [16]:
text_vectorized_train.shape

(70000, 13565)

In [17]:
text_vectorized_test.shape

(30000, 13565)

## Similaridad entre documentos
La similaridad entre documentos es una medida que nos permite cuantificar cuán similares son dos documentos entre sí. Existen varias métricas para calcular la similaridad, pero una de las más comunes es el coseno de la similitud.

El coseno de la similitud se basa en el ángulo entre dos vectores en un espacio vectorial. Se calcula como el coseno del ángulo entre los vectores que representan los documentos. La fórmula para calcular la similaridad del coseno es:
$$\text{Similitud del coseno} = \frac{A \cdot B}{||A|| \cdot ||B||}$$
donde $A$ y $B$ son los vectores que representan los documentos, $(A \cdot B)$ es el producto punto entre los dos vectores, y $||A||$ y $||B||$ son las normas (longitudes) de los vectores.

In [18]:
def cosine_similarity(a, b):
    """Calcula la similitud coseno entre dos vectores"""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

Podemos utilizar la similaridad del coseno para comparar documentos y determinar cuán similares son entre sí. Un valor de similaridad del coseno cercano a 1 indica que los documentos son muy similares, mientras que un valor cercano a 0 indica que son poco similares.

Usemos esta función para generar un buscador en donde podamos buscar documentos similares a un texto de entrada.

In [19]:
def search(query, top_n=5):
    """Busca los textos más similares al texto de entrada"""
    query_vector = count_vectorizer.transform([query]).toarray()[0] # Vectorizamos el texto de entrada
    similarities = []
    for v in text_vectorized_train.toarray():
        similarities.append(cosine_similarity(query_vector, v)) # Calculamos la similitud coseno entre el texto de entrada y cada texto del conjunto de entrenamiento
    #remove nan similarities
    similarities = np.nan_to_num(np.array(similarities)) # Convertimos la lista de similitudes a un array de numpy y reemplazamos los valores NaN por 0
    top_indices = np.argsort(similarities)[-top_n:][::-1] # Obtenemos los índices de los textos más similares
    results = []
    for i in top_indices:
        results.append((spanish_diagnostics["train"]["text"][i], similarities[i])) # Agregamos el texto y su puntaje de similitud
    return results # Retornamos los textos y sus puntajes de similitud

In [20]:
search("paciente con pericoronaritis")

/tmp/ipython-input-831798131.py:3: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


[('P 3,8 PERICORONARITIS', np.float64(1.0)),
 ('PERICORONARITIS PIEZA 3.8-4.8-2.8-1.8', np.float64(1.0)),
 ('PERICORONARITIS PZA 3,8 Y 4,8', np.float64(1.0)),
 ('1.- pericoronaritis', np.float64(1.0)),
 ('Pericoronaritis Pza  4.8    Y 3.8', np.float64(1.0))]

## Spoiler: Pipelines de Sklearn

Los pipelines en la biblioteca scikit-learn son una forma conveniente y eficiente de encadenar múltiples pasos de procesamiento de datos y modelos de aprendizaje automático en un flujo de trabajo coherente. Un pipeline de scikit-learn combina transformadores y estimadores en una secuencia ordenada, donde los datos se pasan secuencialmente a través de cada etapa para su procesamiento.

En scikit-learn, un transformador es cualquier objeto que implementa los métodos fit() y transform(). Los transformadores se utilizan para realizar transformaciones en los datos, como preprocesamiento, extracción de características o reducción de dimensionalidad. Por otro lado, un estimador es cualquier objeto que implementa los métodos fit() y predict(). Los estimadores se utilizan para ajustar modelos a los datos y realizar predicciones.

El pipeline de scikit-learn permite definir una secuencia de pasos, donde cada paso es un par formado por un nombre y un transformador o estimador. Los datos se pasan a través de los pasos en el orden especificado, y cada paso toma los datos de entrada, realiza su operación y pasa los datos transformados al siguiente paso.

In [21]:
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    stop_words = list(stopwords), # Le pasamos la lista de stopwords para eliminarlas del vocabulario
    max_df = 0.05, # Eliminamos del vocabulario el 5% de palabras más frecuentes (stopwords específicas del corpus)
    min_df = 2 # Eliminamos del vocabulario las palabras que tienen una frecuencia menor a 2 (típicamente palabras malformadas)
)

tfidf_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(
    stop_words = list(stopwords),
    max_df = 0.05,
    min_df = 2
)

In [22]:
pipe_bow = sklearn.pipeline.Pipeline([
        ('vectorizer', count_vectorizer),
    ])

In [23]:
pipe_bow.fit_transform(spanish_diagnostics["train"]["text"])

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 372583 stored elements and shape (70000, 13565)>

In [24]:
get_word_scores(spanish_diagnostics["train"]["text"][69983],pipe_bow.named_steps['vectorizer'])

[('caries', np.int64(3)),
 ('profunda', np.int64(2)),
 ('relacion', np.int64(1)),
 ('fundamento', np.int64(1)),
 ('dentinaria', np.int64(1)),
 ('dentina', np.int64(1)),
 ('dental', np.int64(1))]

In [25]:
pipe_tf_idf = sklearn.pipeline.Pipeline([
        ('vectorizer', tfidf_vectorizer),
    ])

In [26]:
pipe_tf_idf.fit_transform(spanish_diagnostics["train"]["text"])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 372583 stored elements and shape (70000, 13565)>

In [27]:
get_word_scores(spanish_diagnostics["train"]["text"][69983],pipe_tf_idf.named_steps['vectorizer'])

[('caries', np.float64(0.5841232685802327)),
 ('profunda', np.float64(0.5052614123771013)),
 ('dentina', np.float64(0.3636207767489411)),
 ('relacion', np.float64(0.30054370920913476)),
 ('dentinaria', np.float64(0.27911008594273823)),
 ('dental', np.float64(0.25831112531154554)),
 ('fundamento', np.float64(0.19062330472026243))]

¿se puede meter mas pasos a un pipeline?

In [28]:
"""
pipe = sklearn.pipeline.Pipeline([
        ('vectorizer', vectorizer), #preprocesamiento
        ('classifier', sklearn.naive_bayes.MultinomialNB()) #clasificador
    ])
"""

"\npipe = sklearn.pipeline.Pipeline([\n        ('vectorizer', vectorizer), #preprocesamiento\n        ('classifier', sklearn.naive_bayes.MultinomialNB()) #clasificador\n    ])\n"